In [156]:
import pandas as pd
import numpy as np
import optuna
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, cross_val_score
import re

/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [157]:
#importazione dei dataset
df = df_california = pd.read_csv("train_titanic.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [158]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [159]:
df = df.drop(columns=['Cabin', 'Name'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(3)
memory usage: 69.7+ KB


In [160]:
def extract_numeric(x):
    if pd.isna(x):
        return np.nan
    s = str(x)
    m = re.findall(r"[-+]?\d*\.?\d+", s)
    if not m:
        return np.nan
    return float(m[0])

numeric_cols_raw = [
    "Ticket",
]

for col in numeric_cols_raw:
    if col in df.columns:
        df[col] = df[col].apply(extract_numeric)
        
df["Age"] = df["Age"].fillna(df["Age"].median())

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       887 non-null    float64
 8   Fare         891 non-null    float64
 9   Embarked     889 non-null    object 
dtypes: float64(3), int64(5), object(2)
memory usage: 69.7+ KB


In [161]:
#rimozione outlier
numeric_cols = df.select_dtypes(include=["float", "int"]).columns.tolist()

for col in numeric_cols:
    Q1 = df[col].quantile(0.2)
    Q3 = df[col].quantile(0.8)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

    df[col] = np.where(df[col].between(lower, upper), df[col], np.nan)

df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 741 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  741 non-null    float64
 1   Survived     741 non-null    float64
 2   Pclass       741 non-null    float64
 3   Sex          741 non-null    object 
 4   Age          741 non-null    float64
 5   SibSp        741 non-null    float64
 6   Parch        741 non-null    float64
 7   Ticket       741 non-null    float64
 8   Fare         741 non-null    float64
 9   Embarked     741 non-null    object 
dtypes: float64(8), object(2)
memory usage: 63.7+ KB


In [162]:
df = pd.get_dummies(df, drop_first=True)

In [163]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000), # Intero tra 100 e 1000
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True), # Scala logaritmica
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0), # % feature per albero
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Reg (Lasso)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Reg (Ridge)
        'n_jobs': 3,
        'random_state': 42
    }
    
    model = xgb.XGBClassifier(**params)
    
    # Usiamo 5-Fold per essere sicuri che il risultato non sia fortuna
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    
    # Restituiamo la media dell'accuratezza
    return scores.mean()

In [164]:
X = df.drop('Survived', axis=1)
y = df["Survived"].astype(int)

In [165]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

**OPTUNA**

In [167]:
study = optuna.create_study(direction='maximize', study_name="XGBoost_BreastCancer")
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n--- Risultati Ottimizzazione ---")
print(f"Miglior Trial (Tentativo #{study.best_trial.number})")
print(f"Accuratezza Migliore (CV): {study.best_value:.4f}")
print("Migliori Iperparametri:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params, n_jobs=3, random_state=42)
final_model.fit(X_train, y_train)
final_acc = final_model.score(X_test, y_test)

print(f"\nAccuratezza Finale sul Test Set: {final_acc:.4f}")

[I 2025-12-05 10:40:12,080] A new study created in memory with name: XGBoost_BreastCancer
Best trial: 0. Best value: 0.795542:   2%|▏         | 1/50 [00:01<01:22,  1.69s/it]

[I 2025-12-05 10:40:13,789] Trial 0 finished with value: 0.795541945591796 and parameters: {'n_estimators': 478, 'learning_rate': 0.026384598345996657, 'max_depth': 8, 'subsample': 0.8980952521502958, 'colsample_bytree': 0.9582511892808679, 'reg_alpha': 2.1997560199819097e-06, 'reg_lambda': 1.161931303621157e-07}. Best is trial 0 with value: 0.795541945591796.


Best trial: 1. Best value: 0.804031:   4%|▍         | 2/50 [00:03<01:33,  1.95s/it]

[I 2025-12-05 10:40:15,921] Trial 1 finished with value: 0.8040307648483122 and parameters: {'n_estimators': 871, 'learning_rate': 0.01688019090234471, 'max_depth': 5, 'subsample': 0.7437291649719286, 'colsample_bytree': 0.6270906384461719, 'reg_alpha': 0.08364037026001644, 'reg_lambda': 0.36784537395015593}. Best is trial 1 with value: 0.8040307648483122.


Best trial: 1. Best value: 0.804031:   6%|▌         | 3/50 [00:05<01:23,  1.78s/it]

...

[I 2025-12-05 10:41:14,335] Trial 47 finished with value: 0.8142287423443954 and parameters: {'n_estimators': 143, 'learning_rate': 0.005846944799440899, 'max_depth': 9, 'subsample': 0.7298424627258034, 'colsample_bytree': 0.8003799016456936, 'reg_alpha': 1.5245776772522993e-05, 'reg_lambda': 0.0003422126878770955}. Best is trial 11 with value: 0.8175758438968808.


Best trial: 11. Best value: 0.817576:  98%|█████████▊| 49/50 [01:03<00:00,  1.08it/s]

[I 2025-12-05 10:41:15,728] Trial 48 finished with value: 0.8108246688505911 and parameters: {'n_estimators': 351, 'learning_rate': 0.003985992600190299, 'max_depth': 10, 'subsample': 0.6842879863399508, 'colsample_bytree': 0.7460091464519131, 'reg_alpha': 0.00010133781111196563, 'reg_lambda': 9.319076406822059e-06}. Best is trial 11 with value: 0.8175758438968808.


Best trial: 11. Best value: 0.817576: 100%|██████████| 50/50 [01:05<00:00,  1.31s/it]


[I 2025-12-05 10:41:17,427] Trial 49 finished with value: 0.8006266913545078 and parameters: {'n_estimators': 532, 'learning_rate': 0.017519786414209655, 'max_depth': 10, 'subsample': 0.6532586834338401, 'colsample_bytree': 0.6129802655555168, 'reg_alpha': 0.23078332492783707, 'reg_lambda': 3.0172612410804696e-08}. Best is trial 11 with value: 0.8175758438968808.

--- Risultati Ottimizzazione ---
Miglior Trial (Tentativo #11)
Accuratezza Migliore (CV): 0.8176
Migliori Iperparametri:
  n_estimators: 417
  learning_rate: 0.0035880393183948724
  max_depth: 10
  subsample: 0.7619962952081557
  colsample_bytree: 0.7254854540741562
  reg_alpha: 1.3836528825820073e-08
  reg_lambda: 1.1932094549027305e-08

Accuratezza Finale sul Test Set: 0.8456
